In [15]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [16]:
import sys
sys.path.insert(1, "../../../")

In [17]:
from train import *
from data_preprocessing import *
from Models.yolov8cls_path import Model

In [18]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [19]:
model = Model(num_classes=10, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=False,
              classifyV8=False,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [20]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [21]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [22]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [23]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [24]:
epochs=3
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

In [26]:
loss_fn = nn.NLLLoss()

In [27]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=epochs-2, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-3/training'),
        record_shapes=True,
        profile_memory=True,
        with_flops=True,
        with_modules=True,
        with_stack=True)


Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [28]:
history, gradient_stats, prof = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, prof, model_path='../../log/YOLOv8cls-version-3/training/models/')

2024-11-18 18:53:11.795806 Epoch 1 
2024-11-18 18:53:18.114568 Batch 1 
[Train] Accuracy: 10.0%, Loss per batch: 2.3226
2024-11-18 18:53:27.642926 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-18 18:53:29.123064 Epoch 2 
2024-11-18 18:53:35.597716 Batch 1 
[Train] Accuracy: 10.0%, Loss per batch: 2.3034
2024-11-18 18:53:44.072715 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-18 18:53:46.663439 Epoch 3 
2024-11-18 18:53:53.065273 Batch 1 
[Train] Accuracy: 10.0%, Loss per batch: 2.2702
2024-11-18 18:54:02.588692 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3027


In [29]:
print(prof.key_averages().table(row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.11%      17.888ms       100.00%       16.315s       16.315s      16.07 Mb           0 b             1            --  
                                          Training Loop         0.03%       4.998ms        52.41%        8.551s        8.551s      16.07 Mb     -17.23 Mb             1            --  
                     Validation Loop Before Data Loader         4.07%     664.47

In [30]:
save_train_outputs('../../log/YOLOv8cls-version-3/training', history, gradient_stats)

TypeError: Object of type DataFrame is not JSON serializable